In [1]:
#!pip install selenium
#!pip install geonamescache
from helper import *
import selenium
import tweepy
import numpy as np
import pandas as pd
import json
from bs4 import BeautifulSoup
from IPython.display import clear_output #clears output
import datetime
import time

API_Key ='irXxqVrXin1KsDRM9g6HdF8p8'
API_Key_Secret = 'sLQvb8voyrTDz5S93vlt5jE5RlMi8xBuOQtSq2pgERe2QVawIC'

Bearer_Token = 'AAAAAAAAAAAAAAAAAAAAACsVlAEAAAAADib49u9Asa6Hbts8UwBEdtc0dsk%3D1d8lVaySeRg1CpJQ6HDLSwK6Mj9wJiptj49PAdwMuvW95zJoar'

Access_Token = '1122492746210660352-COptyz3RtF1AVokHw3027iOdbsZZl1'
Access_Token_Secret = 'slfBz8pcgCrUmewZgSgVKsmIFhfCcFuinAx16q5eh1Vvr'

In [2]:
# authorization of consumer key and consumer secret
auth = tweepy.OAuthHandler(API_Key, API_Key_Secret)
# set access to user's access key and access secret
auth.set_access_token(Access_Token, Access_Token_Secret)
# calling the api
api = tweepy.API(auth)

Client = tweepy.Client(bearer_token=Bearer_Token) 


In [3]:
#tweets_list = api.search_30_day(label = '30day',query = '#blender', fromDate = FROM_DATE,toDate = TO_DATE,maxResults = 10,tweet_mode = "extended")
#tweets_list = tweepy.Cursor(api.search_tweets, "biden", count=100).items(100)


start_date = datetime.datetime.now() - datetime.timedelta(days=14)
end_date = start_date + datetime.timedelta(days=7)

start_date = datetime.datetime.now()
end_date = start_date + datetime.timedelta(days=1)
# Search for tweets within the target date range

tweets_list = []
#ALL_pages = tweepy.Cursor(api.search_tweets, q="since:{} until:{}".format(start_date.strftime("%Y-%m-%d"), end_date.strftime("%Y-%m-%d")), lang="en", tweet_mode="extended", count = 100).pages(50)
tweets_list = tweepy.Cursor(api.search_tweets, q="-filter:replies", lang="en", tweet_mode="extended",include_entities=True,count = 100).items(7000)

#return this if you are using pages again!!!!!
'''
for page in ALL_pages:
    tweets_list.extend(page)
'''
#tweets_list = [t for t in tweets_list if t.in_reply_to_status_id == None]

'\nfor page in ALL_pages:\n    tweets_list.extend(page)\n'

In [4]:
#search_tweets(q, *, geocode, lang, locale, result_type, count, until, since_id, max_id, include_entities)

#user data

DRIVER = webdriver.Chrome()#the chrome

user_name = [];
location = [];
age = [];
gender = [];
followers = [];
following = [];
verified = []
joined = [] #when they joined
tweet_count_user = []

#tweet data
tweet = [];
word_count = [];
lang = []
likes = []
retweet_count = []
is_quote_status = []
date = []
tweet_id = []
hashtags = []
trending_for_area = []

users = [];


for t in tweets_list:
    tweet.append(t.full_text);
    
    users.append(t.user);
    word_count.append(countWordsInString(t.full_text))
    
    lang.append(t.lang)
    likes.append(t.favorite_count) 
    retweet_count.append(t.retweet_count)
    is_quote_status.append(t.is_quote_status)
    date.append(t.created_at.isoformat()[:10])
    tweet_id.append(t.id)
    hashtags.append(t.entities['hashtags'])
    
i = 0
for x in users:
    if i%25 == 0:
        print("waiting")
        time.sleep(6)
    
    j = x._json
    #opens the page and waits for it to load
    screen_name = j['screen_name']
    country = check_country_real(j['location'])
    
    
    DRIVER.get('https://twitter.com/' + screen_name) 
    
    #get the easy stuff
    user_name.append(screen_name)
    following.append(j['friends_count']);
    followers.append(j['followers_count']);
    location.append(country);
    joined.append(x.created_at.isoformat()[:10])
    tweet_count_user.append(x.statuses_count)
    
    trnd = 0
    try:
        trnd = getTrendingTopics(api,country)
    except:
        trnd = []
    
    trending_for_area.append(trnd)
    verified.append(x.verified)
    
    
    
    tmp = getAge(DRIVER)#gets the age
    if(tmp == 2023 - x.created_at.year): #checks if the age is the same as the accounts age
        age.append(np.nan)
    else:
        age.append(tmp)
    
    
    gender.append(getGender(j['description'],j['location'],j['name']));
    
    ##this part simply displays at the bottom how many profiles were checked
    clear_output(wait=True) #clears the output
    
    i+=1
    print(i)

print('done')

7000
done


In [5]:
max = len(users)

In [6]:
dick = {"name":user_name[:max],
        "age":age[:max],
        'location':location[:max],
        "gender":gender[:max],
        'join_date':joined[:max],
        'total tweets': tweet_count_user[:max],
        "followers":followers[:max],
        "following":following[:max],
        'verified' : verified[:max],
        'tweet':tweet[:max],
        'likes':likes[:max],
        'retweets':retweet_count[:max],
        'date':date[:max],
        'word count': word_count[:max],
        'launguage': lang[:max],
        'is quote':is_quote_status[:max],
        'hashtags':hashtags[:max],
        'trending': trending_for_area[:max],
        'tweet id':tweet_id[:max]
       }

df = pd.DataFrame(dick)

In [7]:
df

,name,age,location,gender,join_date,total tweets,followers,following,verified,tweet,likes,retweets,date,word count,launguage,is quote,hashtags,trending,tweet id
0,decomposingg,NaN,None,None,2013-11-28,80978,94,1114,False,RT @JUNlPER: rightwingers: the left is callin...,0,912,2023-01-09,25,en,False,[],None,1612459930421792769
1,Sirius_gul520,NaN,None,None,2020-09-19,59471,2303,194,False,RT @Krystyn31197823: 💫💙💫\n\n... She ... is fro...,0,76,2023-01-09,16,en,False,[],None,1612459930413416448
2,10Danipm,NaN,None,None,2015-05-22,1628,64,510,False,RT @poodle_solana: $PODLE will be airdropping ...,0,28,2023-01-09,21,en,False,[],None,1612459930409132034
3,MasterKitchHell,NaN,None,None,2022-07-16,199963,64,1,False,Where did you season the chicken you idiotic e...,0,0,2023-01-09,11,en,False,[],None,1612459930409107456
4,L1XB3RRY,NaN,None,None,2020-04-14,45807,2439,4771,False,"RT @shwaverse: Thanks God, my ATEEZ is safe🙏🏻✨...",0,41,2023-01-09,8,en,True,[],None,1612459930404929537
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6995,moinurfeelings,NaN,None,None,2010-10-03,107203,1093,618,False,RT @LAYSAINTLAURENT: I encourage people to do ...,0,11,2023-01-09,19,en,False,[],None,1612459894711685122
6996,HibachiStatic,NaN,None,None,2020-04-19,3578,64,58,False,RT @TheeDreadGod: Slaving another 6 day work w...,0,1,2023-01-09,10,en,False,[],None,1612459894707200000
6997,chriscowan1220,NaN,None,None,2021-04-25,1703,695,1901,False,RT @VincentCrypt46: Thank You @elonmusk ! http...,0,41,2023-01-09,7,en,False,[],None,1612459894690414596
6998,ademorh,NaN,None,None,2022-02-18,560,28,197,False,I go carry all the glitters go house check whi...,0,0,2023-01-09,14,en,False,[],None,1612459894686334976


In [8]:
DRIVER.quit()

In [9]:
df.to_csv('first50K.csv')